In [23]:
import tensorflow as tf
from requests import get
import gzip
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Download MNIST 

In [24]:
def download_file(url, file_name):
    with open(file_name, "wb") as file:
        response = get(url)
        file.write(response.content)

In [25]:
def read_mnist(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

In [26]:
download_file('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'MNIST/train-images-idx3-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'MNIST/train-labels-idx1-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 'MNIST/t10k-images-idx3-ubyte.gz')
download_file('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 'MNIST/t10k-labels-idx1-ubyte.gz')

In [27]:
train = {}
test = {}
validation = {}

train['features'], train['labels'] = read_mnist('MNIST/train-images-idx3-ubyte.gz', 'MNIST/train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = read_mnist('MNIST/t10k-images-idx3-ubyte.gz', 'MNIST/t10k-labels-idx1-ubyte.gz')
train['features'], train['labels'] = shuffle(train['features'], train['labels'])
train['features'], validation['features'], train['labels'], validation['labels'] = train_test_split(train['features'], train['labels'], test_size=0.2, random_state=0)

In [28]:
train['features'] = np.pad(train['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
validation['features'] = np.pad(validation['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')
test['features'] = np.pad(test['features'], ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Create virtual concept drift

In [46]:
train['labels']

array([8, 2, 0, ..., 8, 8, 4], dtype=uint8)

# Create concept drift

# Create LeNet

In [43]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
model.add(tf.keras.layers.AveragePooling2D())

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.AveragePooling2D())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(units=120, activation='relu'))
model.add(tf.keras.layers.Dense(units=84, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation = 'softmax'))
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

# Train and Test Model

In [47]:
model.fit(train['features'], train['labels'], batch_size=128, epochs=10, validation_data=(validation['features'], validation['labels']))

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 14s 297us/sample - loss: 0.0105 - accuracy: 0.9970 - val_loss: 0.0685 - val_accuracy: 0.9865
Epoch 2/10
48000/48000 [==============================] - 13s 269us/sample - loss: 0.0079 - accuracy: 0.9973 - val_loss: 0.0785 - val_accuracy: 0.9866
Epoch 3/10
48000/48000 [==============================] - 13s 274us/sample - loss: 0.0071 - accuracy: 0.9977 - val_loss: 0.0922 - val_accuracy: 0.9844
Epoch 4/10
48000/48000 [==============================] - 14s 287us/sample - loss: 0.0063 - accuracy: 0.9979 - val_loss: 0.0845 - val_accuracy: 0.9871
Epoch 5/10
48000/48000 [==============================] - 12s 256us/sample - loss: 0.0071 - accuracy: 0.9980 - val_loss: 0.0971 - val_accuracy: 0.9854
Epoch 6/10
48000/48000 [==============================] - 12s 254us/sample - loss: 0.0065 - accuracy: 0.9980 - val_loss: 0.1355 - val_accuracy: 0.9779
Epoch 7/10
48000/48000 [====================

In [49]:
model.evaluate(test['features'], test['labels'], batch_size=128, verbose=0)

[0.09057514399004883, 0.9891]

In [8]:
model.save("LeNet.h5")